# Access the Spotify API using ```rauth```#

In [1]:
from rauth import OAuth2Service
import json

## Must first establish authorization with Spotify ##

In [2]:
def load_credentials():    
    lines = [line.rstrip('\n') for line in open('secrets_spotify.ini')]        
    for line in lines:
        if "client_id" in line:
            client_id = line.split(': ')[1]
        if "client_secret" in line:
            client_secret = line.split(': ')[1]        
        if "redirect_uri" in line:
            redirect_uri = line.split(': ')[1]

    return client_id, client_secret, redirect_uri

In [3]:
# Spotify
client_id, client_secret, redirect_uri = load_credentials()

In [45]:
# https://rauth.readthedocs.io/en/latest/
# https://developer.spotify.com/web-api/authorization-guide/#authorization-code-flow
spotify = OAuth2Service(
    name='spotify',
    client_id=client_id,
    client_secret=client_secret,    
    access_token_url='https://accounts.spotify.com/api/token',
    authorize_url='https://accounts.spotify.com/authorize',
    base_url='https://api.spotify.com/v1/'
    )

In [5]:
params = {'redirect_uri': redirect_uri, 'response_type': 'code'}
url = spotify.get_authorize_url(**params) # Visit this URL and allow this python app to access your Spotify account

In [7]:
# This code comes from URL returned after following the authorize URL from above
response_code = ''

In [8]:
data = {'code':response_code, 'grant_type': 'authorization_code', 'redirect_uri': redirect_uri}

# I was getting an error about needing a different decoder, adding decoder=json.loads fixed it
session = spotify.get_auth_session(data=data,decoder=json.loads)

## Authorization established, begin accessing the Spotify API ##

#### Profile info for current authorized user ####
https://developer.spotify.com/web-api/console/get-current-user/

In [9]:
session.get('me').json()

{u'display_name': None,
 u'external_urls': {u'spotify': u'https://open.spotify.com/user/johnwmillr'},
 u'followers': {u'href': None, u'total': 0},
 u'href': u'https://api.spotify.com/v1/users/johnwmillr',
 u'id': u'johnwmillr',
 u'images': [],
 u'type': u'user',
 u'uri': u'spotify:user:johnwmillr'}

#### Search for an artist ####
https://developer.spotify.com/web-api/console/get-search-item/

In [36]:
artist='Andy Shauf'
r = session.get('search/?q={artist}&type=artist'.format(artist=artist))
json_obj = r.json()
artist_obj = json_obj['artists']['items'][0]
[key for key in artist_obj.viewkeys()]

[u'genres',
 u'name',
 u'external_urls',
 u'popularity',
 u'uri',
 u'href',
 u'followers',
 u'images',
 u'type',
 u'id']

#### Get the artist's top tracks using the artist ID ####  
https://developer.spotify.com/web-api/console/get-artist/  

In [37]:
artist_id = artist_obj['id']
r = session.get('artists/{id}/top-tracks?country=US'.format(id=artist_id))
json_obj = r.json()
[(track['name'], track['popularity']) for track in json_obj['tracks']]

[(u'The Magician', 51),
 (u'Quite Like You', 51),
 (u'Early to the Party', 48),
 (u'To You', 48),
 (u"You're Out Wasting", 47),
 (u'The Worst in You', 43),
 (u'Twist Your Ankle', 42),
 (u'Begin Again', 41),
 (u'Lick Your Wounds', 41),
 (u'Alexander All Alone', 40)]